# Building the matrices

In [1]:
import numpy as np
A = np.array([
    (1,    0),
    (-2.5, 1)
])
f = np.array((5, 0))
np.linalg.solve(A, f)

array([ 5. , 12.5])

In [2]:
import numpy as np
from scipy.sparse import csr_matrix

data = np.array([1, -2.5, 1])
row_ind = np.array([0, 1, 1])
col_ind = np.array([0, 0, 1])
shape = (2, 2)
matrix = csr_matrix((data, (row_ind, col_ind)), shape=shape)
matrix.todense()

matrix([[ 1. ,  0. ],
        [-2.5,  1. ]])

In [3]:
from bw2calc import spsolve
spsolve(matrix, f)

/Users/cmutel/virtualenvs/bw25_stable/lib/python3.12/site-packages/bw2calc/__init__.py:50: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


array([ 5. , 12.5])

## Indexing in matrices

In [4]:
from matrix_utils import ArrayMapper
import numpy as np

bike_making = 555_555_555  # Easier to read with `_`
steel_tube_making = 666_666_666
bike = 123_456_789
steel_tubes = 111_111
CO2 = 1

am = ArrayMapper(array=np.array([bike_making, steel_tube_making, bike, steel_tubes, CO2]))
am.map_array(np.array([steel_tubes, bike]))

array([1, 2])

In [5]:
from matrix_utils import ArrayMapper
from scipy.sparse import csr_matrix
import numpy as np

bike_making = 555_555_555
steel_tube_making = 666_666_666
bike = 123_456_789
steel_tubes = 111_111
CO2 = 1

process_mapper = ArrayMapper(array=np.array([bike_making, steel_tube_making]))
product_mapper = ArrayMapper(array=np.array([steel_tubes, bike]))
biosphere_mapper = ArrayMapper(array=np.array([CO2]))

data = np.array([1, -2.5, 1])
row_ind = product_mapper.map_array(np.array([bike, steel_tubes, steel_tubes]))
col_ind = process_mapper.map_array(np.array([bike_making, bike_making, steel_tube_making]))
shape = (2, 2)
technosphere_matrix = csr_matrix((data, (row_ind, col_ind)), shape=shape)
technosphere_matrix.todense()

matrix([[-2.5,  1. ],
        [ 1. ,  0. ]])

## Standard LCA Matrices

In [6]:
from matrix_utils import ArrayMapper
from scipy.sparse import csr_matrix
import numpy as np

bike_making = 555_555_555
steel_tube_making = 666_666_666
CO2 = 1

process_mapper = ArrayMapper(array=np.array([bike_making, steel_tube_making]))
biosphere_mapper = ArrayMapper(array=np.array([CO2]))

data = np.array([0.1, 2])
row_ind = biosphere_mapper.map_array(np.array([CO2, CO2]))
col_ind = process_mapper.map_array(np.array([bike_making, steel_tube_making]))
shape = (1, 2)
biosphere_matrix = csr_matrix((data, (row_ind, col_ind)), shape=shape)

data = np.array([1000])
only_ind = biosphere_mapper.map_array(np.array([CO2]))
shape = (1, 1)
characterization_matrix = csr_matrix((data, (only_ind, only_ind)), shape=shape)

biosphere_matrix.todense(), characterization_matrix.todense()

(matrix([[0.1, 2. ]]), matrix([[1000]]))

In [7]:
bike_index = product_mapper.map_array(np.array([bike]))

f = np.zeros(2)
f[bike_index] = 5

characterization_matrix @ biosphere_matrix @ spsolve(technosphere_matrix, f)

array([25500.])

# Datapackages: Persisting Matrix Data

In [9]:
import bw_processing as bwp
bwp.INDICES_DTYPE

[('row', numpy.int64), ('col', numpy.int64)]

In [11]:
import bw_processing as bwp

technosphere_indices = np.array([
    # row, column
    (bike, bike_making),
    (steel_tubes, bike_making),
    (steel_tubes, steel_tube_making)
], dtype=bwp.INDICES_DTYPE)
technosphere_data = np.array([1, -2.5, 1])

dp = bwp.create_datapackage()
dp.add_persistent_vector(
    matrix="technosphere_matrix", 
    indices_array=technosphere_indices,
    data_array=technosphere_data
)

biosphere_indices = np.array([
    (CO2, bike_making),
    (CO2, steel_tube_making),
], dtype=bwp.INDICES_DTYPE)
biosphere_data = np.array([0.1, 2])
dp.add_persistent_vector(
    matrix="biosphere_matrix", 
    indices_array=biosphere_indices,
    data_array=biosphere_data
)

characterization_indices = np.array([
    (CO2, CO2),
], dtype=bwp.INDICES_DTYPE)
characterization_data = np.array([1000.])
dp.add_persistent_vector(
    matrix="characterization_matrix", 
    indices_array=characterization_indices,
    data_array=characterization_data
)

In [13]:
import bw2calc as bc

lca = bc.LCA(demand={bike: 5}, data_objs=[dp])
lca.lci()
lca.lcia()
lca.score

25500.0

In [15]:
lca.characterized_inventory.todense()

matrix([[  500., 25000.]])

In [16]:
from dataclasses import dataclass

@dataclass
class Edge:
    target: int
    source: int
    data: dict

In [17]:
from dataclasses import dataclass
from enum import StrEnum

@dataclass
class Node:
    id: int
    type: StrEnum
    collection: str
    data: dict

In [18]:
from dataclasses import dataclass
from datetime import datetime
from typing import Any

@dataclass
class Collection:
    id: Any
    modified: datetime
    data: dict